In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder


# Zadanie 1:  

## Email Spam

 Masz dane o 12 emailach z informacją czy to spam czy nie:

 **Zadania do wykonania:**

**a) Ręczne obliczenia**
1. Oblicz prawdopodobieństwa a priori: P(Spam=TAK) i P(Spam=NIE)
2. Dla każdej cechy oblicz prawdopodobieństwa warunkowe
3. Przewidź klasę dla nowego emaila:
```
   Słowo_1 = 'darmowy'
   Słowo_2 = 'wygrana'  
   Wykrzyknik = 'TAK'
```

Oblicz prawdopodobieństwa dla obu klas (TAK lub NIE) i znormalizuj

**b) Implementacja w Python**

1. Zaimplementuj obliczenia z punktu a) w Python (bez sklearn)
2. Porównaj wyniki z ręcznymi obliczeniami

**c) Sklearn**

1. Użyj `CategoricalNB` z sklearn do wytrenowania modelu
2. Porównaj wyniki z własnymi obliczeniami
3. Wyjaśnij różnice (jeśli są)

In [ ]:
data_spam = {
    'Słowo_1': ['darmowy', 'darmowy', 'spotkanie', 'raport', 'oferta', 'darmowy',
                'spotkanie', 'oferta', 'raport', 'darmowy', 'spotkanie', 'oferta'],
    'Słowo_2': ['wygrana', 'wygrana', 'jutro', 'kwartalny', 'specjalna', 'rabat',
                'dziś', 'limitowana', 'miesięczny', 'rabat', 'pilne', 'wyjątkowa'],
    'Wykrzyknik': ['TAK', 'TAK', 'NIE', 'NIE', 'TAK', 'TAK',
                   'NIE', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK'],
    'Spam': ['TAK', 'TAK', 'NIE', 'NIE', 'NIE', 'TAK',
             'NIE', 'NIE', 'NIE', 'TAK', 'NIE', 'NIE']
}

df = pd.DataFrame(data_spam)
print(df.to_string(index=True))

      Słowo_1     Słowo_2 Wykrzyknik Spam
0     darmowy     wygrana        TAK  TAK
1     darmowy     wygrana        TAK  TAK
2   spotkanie       jutro        NIE  NIE
3      raport   kwartalny        NIE  NIE
4      oferta   specjalna        TAK  NIE
5     darmowy       rabat        TAK  TAK
6   spotkanie        dziś        NIE  NIE
7      oferta  limitowana        TAK  NIE
8      raport  miesięczny        NIE  NIE
9     darmowy       rabat        TAK  TAK
10  spotkanie       pilne        NIE  NIE
11     oferta   wyjątkowa        TAK  NIE


In [ ]:
zlicz = df['Spam'].value_counts()
priors =  zlicz / len(df)

print(f"Liczba przypadków TAK: {zlicz['TAK']}")
print(f"Liczba przypadków NIE: {zlicz['NIE']}")
print(f"Razem: {len(df)}")
print()
print(f"P(SPAM = TAK) = {zlicz['TAK']}/{len(df)} = {priors['TAK']:.3f}")
print(f"P(SPAM = NIE) = {zlicz['NIE']}/{len(df)} = {priors['NIE']:.3f}")
print()


Liczba przypadków TAK: 4
Liczba przypadków NIE: 8
Razem: 12

P(SPAM = TAK) = 4/12 = 0.333
P(SPAM = NIE) = 8/12 = 0.667



In [ ]:
cechy = ['Słowo_1','Słowo_2','Wykrzyknik']
warunki = {}

for cecha in cechy:
  warunki[cecha] = {}

  for czy in ['TAK','NIE']:
    klasa_dane = df[df['Spam'] == czy]
    klasa_ile = len(klasa_dane)

    print(f"\nDla klasy {czy} ({klasa_ile} przypadków):")

    warunki[cecha][czy] = {}
    cecha_ile = klasa_dane[cecha].value_counts()

    for cecha_wart in df[cecha].unique():
      liczba = cecha_ile.get(cecha_wart, 0)
      p = liczba / klasa_ile
      warunki[cecha][czy][cecha_wart] = p

      print(f"{cecha_wart:12s}: {liczba} przypadków - P({cecha}={cecha_wart} | {czy}) = {liczba}/{klasa_ile} = {p:.3f}")



Dla klasy TAK (4 przypadków):
darmowy     : 4 przypadków - P(Słowo_1=darmowy | TAK) = 4/4 = 1.000
spotkanie   : 0 przypadków - P(Słowo_1=spotkanie | TAK) = 0/4 = 0.000
raport      : 0 przypadków - P(Słowo_1=raport | TAK) = 0/4 = 0.000
oferta      : 0 przypadków - P(Słowo_1=oferta | TAK) = 0/4 = 0.000

Dla klasy NIE (8 przypadków):
darmowy     : 0 przypadków - P(Słowo_1=darmowy | NIE) = 0/8 = 0.000
spotkanie   : 3 przypadków - P(Słowo_1=spotkanie | NIE) = 3/8 = 0.375
raport      : 2 przypadków - P(Słowo_1=raport | NIE) = 2/8 = 0.250
oferta      : 3 przypadków - P(Słowo_1=oferta | NIE) = 3/8 = 0.375

Dla klasy TAK (4 przypadków):
wygrana     : 2 przypadków - P(Słowo_2=wygrana | TAK) = 2/4 = 0.500
jutro       : 0 przypadków - P(Słowo_2=jutro | TAK) = 0/4 = 0.000
kwartalny   : 0 przypadków - P(Słowo_2=kwartalny | TAK) = 0/4 = 0.000
specjalna   : 0 przypadków - P(Słowo_2=specjalna | TAK) = 0/4 = 0.000
rabat       : 2 przypadków - P(Słowo_2=rabat | TAK) = 2/4 = 0.500
dziś        : 0 przypad

In [ ]:
for cecha in cechy:
  print(f"{cecha}:")
  print(f"{'Wartość':15s} | {f"P({cecha}|TAK)":20s} | {f"P({cecha}|NIE)":20s}")
  print()
  for wart in df[cecha].unique():
    print(f"{wart:15s} | {warunki[cecha]['TAK'].get(wart,0):20.4f} | {warunki[cecha]['NIE'].get(wart,0):20.4f}")
  print()

Słowo_1:
Wartość         | P(Słowo_1|TAK)       | P(Słowo_1|NIE)      

darmowy         |               1.0000 |               0.0000
spotkanie       |               0.0000 |               0.3750
raport          |               0.0000 |               0.2500
oferta          |               0.0000 |               0.3750

Słowo_2:
Wartość         | P(Słowo_2|TAK)       | P(Słowo_2|NIE)      

wygrana         |               0.5000 |               0.0000
jutro           |               0.0000 |               0.1250
kwartalny       |               0.0000 |               0.1250
specjalna       |               0.0000 |               0.1250
rabat           |               0.5000 |               0.0000
dziś            |               0.0000 |               0.1250
limitowana      |               0.0000 |               0.1250
miesięczny      |               0.0000 |               0.1250
pilne           |               0.0000 |               0.1250
wyjątkowa       |               0.0000 |         

In [ ]:
nowy = {'Słowo_1':'darmowy', 'Słowo_2':'wygrana', 'Wykrzyknik':'TAK'}

wyniki = {}

for czy in ['TAK', 'NIE']:
  wynik = priors[czy]
  print(f"Prior: P({czy}) = {wynik:.3f}")

  for klasa, wart in nowy.items():
    print(f"P({klasa}={wart} | {czy}) = {warunki[klasa][czy].get(wart,0):.3f}")
    wynik *= warunki[klasa][czy].get(wart,0)
    print(f"Po mnożeniu: {wynik:.6f}")
    print()

  print(f"Wynik końcowy dla {czy}: {wynik:.6f}")
  print()
  wyniki[czy] = wynik

Prior: P(TAK) = 0.333
P(Słowo_1=darmowy | TAK) = 1.000
Po mnożeniu: 0.333333

P(Słowo_2=wygrana | TAK) = 0.500
Po mnożeniu: 0.166667

P(Wykrzyknik=TAK | TAK) = 1.000
Po mnożeniu: 0.166667

Wynik końcowy dla TAK: 0.166667

Prior: P(NIE) = 0.667
P(Słowo_1=darmowy | NIE) = 0.000
Po mnożeniu: 0.000000

P(Słowo_2=wygrana | NIE) = 0.000
Po mnożeniu: 0.000000

P(Wykrzyknik=TAK | NIE) = 0.375
Po mnożeniu: 0.000000

Wynik końcowy dla NIE: 0.000000



In [ ]:
suma = sum(wyniki.values())
normalizacja = {k: v/suma for k, v in wyniki.items()}

print("Prawdopodobieństwo po normalizacji:")
for czy in ['TAK', 'NIE']:
  procent = normalizacja[czy]*100
  print(f"P({czy}) = {wyniki[czy]:.6f} / ({suma:.2f}) = {procent:.1f}%")

print("Rekord:\n")
for key in nowy:
  print(f"{key}: {nowy[key]}")
pred = max(normalizacja, key=normalizacja.get)
print(f"\nPREDYKCJA: {pred}")
print(f"Pewność: {normalizacja[pred] * 100:.1f}%")

Prawdopodobieństwo po normalizacji:
P(TAK) = 0.166667 / (0.17) = 100.0%
P(NIE) = 0.000000 / (0.17) = 0.0%
Rekord:

Słowo_1: darmowy
Słowo_2: wygrana
Wykrzyknik: TAK

PREDYKCJA: TAK
Pewność: 100.0%


In [ ]:
slownik = {}
cechy = ['Słowo_1', 'Słowo_2', 'Wykrzyknik']
X_encoded = df[cechy].copy()

for cecha in cechy:
  le = LabelEncoder()
  X_encoded[cecha] = le.fit_transform(df[cecha])
  slownik[cecha] = le

cel = LabelEncoder()
y_encoded = cel.fit_transform(df['Spam'])

model = CategoricalNB(alpha = 1.0)
model.fit(X_encoded, y_encoded)

nowy = {'Słowo_1':'darmowy', 'Słowo_2':'wygrana', 'Wykrzyknik':'TAK'}

nowy_encoded = []
for cecha in cechy:
  wart = nowy[cecha]
  encoded_wart = slownik[cecha].transform([wart])[0]
  nowy_encoded.append(encoded_wart)

nowy_array = np.array([nowy_encoded])

pred = model.predict(nowy_array)
prob = model.predict_proba(nowy_array)

print("SKLEARN")
print(f"Predykcja: {cel.inverse_transform(pred)[0]}")
print(f"Prawdopodobieństwa: TAK = {prob[0][1]:.3f}, NIE={prob[0][0]:.3f}")

SKLEARN
Predykcja: TAK
Prawdopodobieństwa: TAK = 0.968, NIE=0.032


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


Różnice wynikają z zerowych prawdpodobieństw otrzymanych przed użyciem sklearn.